# Exploratory Data Analysis: F1 Race Strategy Data

This notebook loads the F1 race data before and after the transformation pipeline to inspect the features and distributions on the Jolpica F1 datasource.


Schema diagram:  https://dbdocs.io/jolpica/jolpica-f1?view=relationships

Obtaining CSVs:
- https://github.com/jolpica/jolpica-f1/discussions/261
- https://api.jolpi.ca/data/dumps/download/
- https://api.jolpi.ca/data/dumps/download/delayed/?dump_type=csv

**Note:** Ensure your environment is set up and database connectivity is configured.

In [1]:
import sys
import os
import pandas as pd
import numpy as np

# Add project root to path to access modules
# Assuming this notebook is in the 'notebooks' directory
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

print(f"Project root added to path: {project_root}")

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

Project root added to path: /home/freesample/projects/github.com/modestprophet/gonzo-pit-strategy


## 1. Load Raw Data (Before Pipeline)

We load the raw data directly from the repository using `F1DataRepository.get_all_race_history()`.

In [2]:
from db.repositories.data_repository import F1DataRepository

# Load raw data
print("Fetching raw data from repository...")
df_raw = F1DataRepository.get_all_race_history()

print(f"Raw data shape: {df_raw.shape}")
display(df_raw.head())

# Basic info
df_raw.info()

output_dir = os.path.join(project_root, 'data', 'processed')
os.makedirs(output_dir, exist_ok=True)

# Export raw data
raw_output_path = os.path.join(output_dir, 'raw.csv')
df_raw.to_csv(raw_output_path, index=False)
print(f"✓ Raw data exported to: {raw_output_path}")
print(f"  Shape: {df_raw.shape}")

Fetching raw data from repository...
2026-01-12 14:31:30,886 - SECURITY - INFO - Vault client initialized successfully
2026-01-12 14:31:30,918 - db.connection_pool - INFO - Creating database engine for postgresql+psycopg2 at 10.0.20.18:5432/f1db
Raw data shape: (25873, 30)


,round_id,round_number,round_name,race_date,circuit_id,circuit_name,country,race_year,season_url,team_id,team_name,driver_id,driver_code,forename,surname,qualifying_position,qualifying_time,starting_grid,finish_position,status,race_time,points,laps_completed,fastest_lap_rank,driver_total_points,driver_championship_position,driver_wins,team_total_points,team_championship_position,team_wins
0,1,1,British Grand Prix,1950-05-13,1,Silverstone Circuit,UK,1950,https://en.wikipedia.org/wiki/1950_Formula_One...,1,Alfa Romeo,1,None,Nino,Farina,None,None,1,1,0,02:13:23.6,9.0,70,NaN,9.0,1.0,1,0.0,NaN,1.0
1,1,1,British Grand Prix,1950-05-13,1,Silverstone Circuit,UK,1950,https://en.wikipedia.org/wiki/1950_Formula_One...,1,Alfa Romeo,2,None,Luigi,Fagioli,None,None,2,2,0,02:13:26.2,6.0,70,NaN,6.0,2.0,0,0.0,NaN,1.0
2,1,1,British Grand Prix,1950-05-13,1,Silverstone Circuit,UK,1950,https://en.wikipedia.org/wiki/1950_Formula_One...,1,Alfa Romeo,3,None,Reg,Parnell,None,None,4,3,0,02:14:15.6,4.0,70,NaN,4.0,3.0,0,0.0,NaN,1.0
3,1,1,British Grand Prix,1950-05-13,1,Silverstone Circuit,UK,1950,https://en.wikipedia.org/wiki/1950_Formula_One...,5,Talbot-Lago,4,None,Yves,Cabantous,None,None,6,4,1,None,3.0,68,NaN,3.0,4.0,0,0.0,NaN,0.0
4,1,1,British Grand Prix,1950-05-13,1,Silverstone Circuit,UK,1950,https://en.wikipedia.org/wiki/1950_Formula_One...,5,Talbot-Lago,5,None,Louis,Rosier,None,None,9,5,1,None,2.0,68,NaN,2.0,5.0,0,0.0,NaN,0.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25873 entries, 0 to 25872
Data columns (total 30 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   round_id                      25873 non-null  int64  
 1   round_number                  25873 non-null  int64  
 2   round_name                    25873 non-null  object 
 3   race_date                     25873 non-null  object 
 4   circuit_id                    25873 non-null  int64  
 5   circuit_name                  25873 non-null  object 
 6   country                       25873 non-null  object 
 7   race_year                     25873 non-null  int64  
 8   season_url                    25873 non-null  object 
 9   team_id                       25873 non-null  int64  
 10  team_name                     25873 non-null  object 
 11  driver_id                     25873 non-null  int64  
 12  driver_code                   10644 non-null  object 
 13  f

### Raw Data Description

In [ ]:
display(df_raw.describe(include='all'))

## 2. Process Data (Pipeline Execution)

We initialize and run the `DataPipeline` to transform the data. This will apply all steps defined in `config/pipeline_race_history.json`.

In [3]:
from training.data_pipeline import DataPipeline
from config.config import config

# Initialize pipeline
# config_name='pipeline' maps to pipeline_race_history.json as per config/config.py
pipeline = DataPipeline(config_name="pipeline") 

# Run the pipeline
# pipeline.process() fetches data internally and applies transformations
print("Running data pipeline...")
df_processed = pipeline.process()

print(f"Processed data shape: {df_processed.shape}")
display(df_processed.head())

2026-01-12 14:31:47,220 - training.data_pipeline - DEBUG - Added step: null_value_cleaner
2026-01-12 14:31:47,221 - training.data_pipeline - DEBUG - Added step: data_type_converter
2026-01-12 14:31:47,222 - training.data_pipeline - DEBUG - Added step: qualifying_time_converter
2026-01-12 14:31:47,223 - training.data_pipeline - DEBUG - Added step: categorical_encoder
2026-01-12 14:31:47,223 - training.data_pipeline - DEBUG - Added step: numerical_scaler
2026-01-12 14:31:47,224 - training.data_pipeline - DEBUG - Added step: lagged_feature_generator
2026-01-12 14:31:47,225 - training.data_pipeline - DEBUG - Added step: season_progress
2026-01-12 14:31:47,226 - training.data_pipeline - DEBUG - Added step: z_score_clipper
2026-01-12 14:31:47,227 - training.data_pipeline - DEBUG - Added step: drop_columns
2026-01-12 14:31:47,228 - training.data_pipeline - INFO - Initialized f1_race_data_pipeline pipeline v0.2.0 with 9 steps
Running data pipeline...
2026-01-12 14:31:47,230 - training.data_pip

/home/freesample/projects/github.com/modestprophet/gonzo-pit-strategy/training/pipeline_steps/null_cleaner.py:71: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna(value)


2026-01-12 14:31:48,392 - training.pipeline_steps.encoders - DEBUG - Encoded team_name into 202 features
2026-01-12 14:31:48,608 - training.pipeline_steps.encoders - DEBUG - Encoded driver_code into 100 features
2026-01-12 14:31:48,648 - training.pipeline_steps.encoders - DEBUG - Dropped original columns: ['circuit_name', 'team_name', 'driver_code']
2026-01-12 14:31:48,657 - training.data_pipeline - INFO - Shape after CategoricalEncoder: (25873, 407)
2026-01-12 14:31:48,659 - training.data_pipeline - DEBUG - Cols after step: ['round_id', 'round_number', 'round_name', 'race_date', 'circuit_id', 'country', 'race_year', 'season_url', 'team_id', 'driver_id', 'forename', 'surname', 'qualifying_position', 'qualifying_time', 'starting_grid', 'finish_position', 'status', 'race_time', 'points', 'laps_completed', 'fastest_lap_rank', 'driver_total_points', 'driver_championship_position', 'driver_wins', 'team_total_points', 'team_championship_position', 'team_wins', 'qualifying_time_missing', 'qua

,round_name,country,qualifying_position,starting_grid,finish_position,race_time,laps_completed,fastest_lap_rank,qualifying_time_missing,qualifying_time_seconds,circuit_name_Adelaide Street Circuit,circuit_name_Ain Diab,circuit_name_Aintree,circuit_name_Albert Park Grand Prix Circuit,circuit_name_Autodromo Enzo e Dino Ferrari,circuit_name_Autodromo Internazionale del Mugello,circuit_name_Autodromo Nazionale di Monza,circuit_name_Autódromo Hermanos Rodríguez,circuit_name_Autódromo Internacional Nelson Piquet,circuit_name_Autódromo Internacional do Algarve,circuit_name_Autódromo José Carlos Pace,circuit_name_Autódromo Juan y Oscar Gálvez,circuit_name_Autódromo do Estoril,circuit_name_Bahrain International Circuit,circuit_name_Baku City Circuit,circuit_name_Brands Hatch,circuit_name_Buddh International Circuit,circuit_name_Charade Circuit,circuit_name_Circuit Bremgarten,circuit_name_Circuit Gilles Villeneuve,circuit_name_Circuit Mont-Tremblant,circuit_name_Circuit Park Zandvoort,circuit_name_Circuit Paul Ricard,circuit_name_Circuit de Barcelona-Catalunya,circuit_name_Circuit de Monaco,circuit_name_Circuit de Nevers Magny-Cours,circuit_name_Circuit de Pedralbes,circuit_name_Circuit de Spa-Francorchamps,circuit_name_Circuit of the Americas,circuit_name_Circuito da Boavista,circuit_name_Circuito de Jerez,circuit_name_Detroit Street Circuit,circuit_name_Dijon-Prenois,circuit_name_Donington Park,circuit_name_Fair Park,circuit_name_Fuji Speedway,circuit_name_Hockenheimring,circuit_name_Hungaroring,circuit_name_Indianapolis Motor Speedway,circuit_name_Istanbul Park,circuit_name_Jarama,circuit_name_Jeddah Corniche Circuit,circuit_name_Korean International Circuit,circuit_name_Kyalami,circuit_name_Las Vegas Street Circuit,circuit_name_Las Vegas Strip Street Circuit,circuit_name_Le Mans,circuit_name_Long Beach,circuit_name_Losail International Circuit,circuit_name_Marina Bay Street Circuit,circuit_name_Miami International Autodrome,circuit_name_Monsanto Park Circuit,circuit_name_Montjuïc,circuit_name_Mosport International Raceway,circuit_name_Nivelles-Baulers,circuit_name_Nürburgring,circuit_name_Okayama International Circuit,circuit_name_Pescara Circuit,circuit_name_Phoenix street circuit,circuit_name_Prince George Circuit,circuit_name_Red Bull Ring,circuit_name_Reims-Gueux,circuit_name_Riverside International Raceway,circuit_name_Rouen-Les-Essarts,circuit_name_Scandinavian Raceway,circuit_name_Sebring International Raceway,circuit_name_Sepang International Circuit,circuit_name_Shanghai International Circuit,circuit_name_Silverstone Circuit,circuit_name_Sochi Autodrom,circuit_name_Suzuka Circuit,circuit_name_Valencia Street Circuit,circuit_name_Watkins Glen,circuit_name_Yas Marina Circuit,circuit_name_Zeltweg,circuit_name_Zolder,team_name_AGS,team_name_ATS,team_name_Adams,team_name_Alfa Romeo,team_name_AlphaTauri,team_name_Alpine F1 Team,team_name_Alta,team_name_Amon,team_name_Andrea Moda,team_name_Arrows,team_name_Arzani-Volpini,team_name_Aston Butterworth,team_name_Aston Martin,team_name_BAR,team_name_BMW,team_name_BMW Sauber,team_name_BRM,team_name_BRM-Ford,team_name_BRP,team_name_Behra-Porsche,team_name_Bellasi,team_name_Benetton,team_name_Boro,team_name_Brabham,team_name_Brabham-Alfa Romeo,team_name_Brabham-BRM,team_name_Brabham-Climax,team_name_Brabham-Ford,team_name_Brabham-Repco,team_name_Brawn,team_name_Bromme,team_name_Bugatti,team_name_Caterham,team_name_Christensen,team_name_Coloni,team_name_Connaught,team_name_Connew,team_name_Cooper,team_name_Cooper-ATS,team_name_Cooper-Alfa Romeo,team_name_Cooper-BRM,team_name_Cooper-Borgward,team_name_Cooper-Castellotti,team_name_Cooper-Climax,team_name_Cooper-Ferrari,team_name_Cooper-Maserati,team_name_Cooper-OSCA,team_name_Dallara,team_name_De Tomaso,team_name_De Tomaso-Alfa Romeo,team_name_De Tomaso-Osca,team_name_Deidt,team_name_Del Roy,team_name_Derrington,team_name_Dunn,team_name_EMW,team_name_ENB,team_name_ERA,team_name_Eagle-Climax,team_name_Eagle-Weslake,team_name_Elder,team_name_E

### Processed Data Description

In [4]:
display(df_processed.describe(include='all'))
df_processed.info()

,round_name,country,qualifying_position,starting_grid,finish_position,race_time,laps_completed,fastest_lap_rank,qualifying_time_missing,qualifying_time_seconds,circuit_name_Adelaide Street Circuit,circuit_name_Ain Diab,circuit_name_Aintree,circuit_name_Albert Park Grand Prix Circuit,circuit_name_Autodromo Enzo e Dino Ferrari,circuit_name_Autodromo Internazionale del Mugello,circuit_name_Autodromo Nazionale di Monza,circuit_name_Autódromo Hermanos Rodríguez,circuit_name_Autódromo Internacional Nelson Piquet,circuit_name_Autódromo Internacional do Algarve,circuit_name_Autódromo José Carlos Pace,circuit_name_Autódromo Juan y Oscar Gálvez,circuit_name_Autódromo do Estoril,circuit_name_Bahrain International Circuit,circuit_name_Baku City Circuit,circuit_name_Brands Hatch,circuit_name_Buddh International Circuit,circuit_name_Charade Circuit,circuit_name_Circuit Bremgarten,circuit_name_Circuit Gilles Villeneuve,circuit_name_Circuit Mont-Tremblant,circuit_name_Circuit Park Zandvoort,circuit_name_Circuit Paul Ricard,circuit_name_Circuit de Barcelona-Catalunya,circuit_name_Circuit de Monaco,circuit_name_Circuit de Nevers Magny-Cours,circuit_name_Circuit de Pedralbes,circuit_name_Circuit de Spa-Francorchamps,circuit_name_Circuit of the Americas,circuit_name_Circuito da Boavista,circuit_name_Circuito de Jerez,circuit_name_Detroit Street Circuit,circuit_name_Dijon-Prenois,circuit_name_Donington Park,circuit_name_Fair Park,circuit_name_Fuji Speedway,circuit_name_Hockenheimring,circuit_name_Hungaroring,circuit_name_Indianapolis Motor Speedway,circuit_name_Istanbul Park,circuit_name_Jarama,circuit_name_Jeddah Corniche Circuit,circuit_name_Korean International Circuit,circuit_name_Kyalami,circuit_name_Las Vegas Street Circuit,circuit_name_Las Vegas Strip Street Circuit,circuit_name_Le Mans,circuit_name_Long Beach,circuit_name_Losail International Circuit,circuit_name_Marina Bay Street Circuit,circuit_name_Miami International Autodrome,circuit_name_Monsanto Park Circuit,circuit_name_Montjuïc,circuit_name_Mosport International Raceway,circuit_name_Nivelles-Baulers,circuit_name_Nürburgring,circuit_name_Okayama International Circuit,circuit_name_Pescara Circuit,circuit_name_Phoenix street circuit,circuit_name_Prince George Circuit,circuit_name_Red Bull Ring,circuit_name_Reims-Gueux,circuit_name_Riverside International Raceway,circuit_name_Rouen-Les-Essarts,circuit_name_Scandinavian Raceway,circuit_name_Sebring International Raceway,circuit_name_Sepang International Circuit,circuit_name_Shanghai International Circuit,circuit_name_Silverstone Circuit,circuit_name_Sochi Autodrom,circuit_name_Suzuka Circuit,circuit_name_Valencia Street Circuit,circuit_name_Watkins Glen,circuit_name_Yas Marina Circuit,circuit_name_Zeltweg,circuit_name_Zolder,team_name_AGS,team_name_ATS,team_name_Adams,team_name_Alfa Romeo,team_name_AlphaTauri,team_name_Alpine F1 Team,team_name_Alta,team_name_Amon,team_name_Andrea Moda,team_name_Arrows,team_name_Arzani-Volpini,team_name_Aston Butterworth,team_name_Aston Martin,team_name_BAR,team_name_BMW,team_name_BMW Sauber,team_name_BRM,team_name_BRM-Ford,team_name_BRP,team_name_Behra-Porsche,team_name_Bellasi,team_name_Benetton,team_name_Boro,team_name_Brabham,team_name_Brabham-Alfa Romeo,team_name_Brabham-BRM,team_name_Brabham-Climax,team_name_Brabham-Ford,team_name_Brabham-Repco,team_name_Brawn,team_name_Bromme,team_name_Bugatti,team_name_Caterham,team_name_Christensen,team_name_Coloni,team_name_Connaught,team_name_Connew,team_name_Cooper,team_name_Cooper-ATS,team_name_Cooper-Alfa Romeo,team_name_Cooper-BRM,team_name_Cooper-Borgward,team_name_Cooper-Castellotti,team_name_Cooper-Climax,team_name_Cooper-Ferrari,team_name_Cooper-Maserati,team_name_Cooper-OSCA,team_name_Dallara,team_name_De Tomaso,team_name_De Tomaso-Alfa Romeo,team_name_De Tomaso-Osca,team_name_Deidt,team_name_Del Roy,team_name_Derrington,team_name_Dunn,team_name_EMW,team_name_ENB,team_name_ERA,team_name_Eagle-Climax,team_name_Eagle-Weslake,team_name_Elder,team_name_E

<class 'pandas.core.frame.DataFrame'>
Index: 25873 entries, 0 to 25869
Columns: 396 entries, round_name to season_progress_percent
dtypes: float64(391), int64(2), object(3)
memory usage: 78.4+ MB


## 3. Comparison / Changelog

Compare the feature set before and after transformation.

In [5]:
raw_cols = set(df_raw.columns)
proc_cols = set(df_processed.columns)

added_cols = proc_cols - raw_cols
removed_cols = raw_cols - proc_cols

print(f"\nColumns Added ({len(added_cols)}):\n{sorted(list(added_cols))}")
print(f"\nColumns Removed ({len(removed_cols)}):\n{sorted(list(removed_cols))}")

print("\nCommon Columns processed:")
common_cols = raw_cols.intersection(proc_cols)
print(sorted(list(common_cols)))


Columns Added (388):
['circuit_name_Adelaide Street Circuit', 'circuit_name_Ain Diab', 'circuit_name_Aintree', 'circuit_name_Albert Park Grand Prix Circuit', 'circuit_name_Autodromo Enzo e Dino Ferrari', 'circuit_name_Autodromo Internazionale del Mugello', 'circuit_name_Autodromo Nazionale di Monza', 'circuit_name_Autódromo Hermanos Rodríguez', 'circuit_name_Autódromo Internacional Nelson Piquet', 'circuit_name_Autódromo Internacional do Algarve', 'circuit_name_Autódromo José Carlos Pace', 'circuit_name_Autódromo Juan y Oscar Gálvez', 'circuit_name_Autódromo do Estoril', 'circuit_name_Bahrain International Circuit', 'circuit_name_Baku City Circuit', 'circuit_name_Brands Hatch', 'circuit_name_Buddh International Circuit', 'circuit_name_Charade Circuit', 'circuit_name_Circuit Bremgarten', 'circuit_name_Circuit Gilles Villeneuve', 'circuit_name_Circuit Mont-Tremblant', 'circuit_name_Circuit Park Zandvoort', 'circuit_name_Circuit Paul Ricard', 'circuit_name_Circuit de Barcelona-Catalunya'

## Export

In [6]:
# Export processed data
processed_output_path = os.path.join(output_dir, 'processed.csv')
df_processed.to_csv(processed_output_path, index=False)
print(f"✓ Processed data exported to: {processed_output_path}")
print(f"  Shape: {df_processed.shape}")

✓ Raw data exported to: /home/freesample/projects/github.com/modestprophet/gonzo-pit-strategy/data/processed/raw.csv
  Shape: (25873, 30)
✓ Processed data exported to: /home/freesample/projects/github.com/modestprophet/gonzo-pit-strategy/data/processed/processed.csv
  Shape: (25873, 396)
